In [ ]:
from __future__ import division, unicode_literals, print_function  # for compatibility with Python 2 and 3
import sys
sys.path.append('../src')  # Adjust the path as necessary
# sys.path.append('..')  # Adjust the path as necessary

# from SPTnano import ROISelector, process_directory, generate_file_tree, display_file_tree, overlay_tracks_with_movie, plot_histograms,  read_mat_file, add_microns_and_secs, add_unique_id, plot_particle_trajectory, plot_multiple_particles, filter_stubs
import SPTnano as spt
# from SPTnano import ROISelector, process_directory

master = spt.config.MASTER
saved_data = spt.config.SAVED_DATA

# import pixel size and frame rate
pixelsize_microns = spt.config.PIXELSIZE_MICRONS
time_between_frames = spt.config.TIME_BETWEEN_FRAMES
orderofconditions = spt.config.ORDEROFCONDITIONS
features = spt.config.FEATURES

min_track_length = spt.config.TIME_WINDOW

from IPython.display import Markdown, display

import matplotlib as mpl
import matplotlib.pyplot as plt


import numpy as np
import pandas as pd
from pandas import DataFrame, Series  # for convenience

import pims
import trackpy as tp
import os
import glob
import nd2
import seaborn as sns

# change the following to %matplotlib notebook for interactive plotting
%matplotlib inline

# Optionally, tweak styles.
mpl.rc('figure',  figsize=(10, 5))
mpl.rc('image', cmap='gray')

sns.set_context("notebook", rc={"xtick.labelsize": 10, "ytick.labelsize": 10})


In [ ]:
from SPTnano.batch_roi_selector import ROISelector  # Adjust 'your_module' to your actual module name

# Paths


input_directory = master  # Directory where your actual ND2/TIFF image data is stored
# output_directory = 'D:/Neurons_HTT_September_2024/9-11-2024_neurons_100x_25perc_10ms_20H20Scortandvent_150H20Scortandvent_analyze10'  # Directory to save the cropped images
# output_directory = 'D:/PLURIPOTENT_KINESIN_JULY_2024/10ms_60x_kinesin_pluripotent_priorityone_analyze' 
# output_directory = 'D:/2_5_2025and2_7_2025_corticalneurons_analyze'
# output_directory = 'D:/2_25_2025_CorticalNeuron_20H20S_FreeHalo_20H77S_77H20S_analyze'

# output_directory = 'D:/2_21_2025_KinesinEScells_RUES2_HTT72CAG_HTTKO_10ms_analyze'

# output_directory = 'D:/3_4_2025_CorticalNeuron_20H20S_freehalo_20H77S_77H20S_analyze'

# output_directory = 'D:/3_11_2025_VentralNeuron_20H20S_freehalo_77H20S_20H77S_analyze'
# output_directory = 'D:/3_13_2025_KinesininES_TN_RUES2_CAG72_KO_10ms_analyze'
# output_directory = 'D:/3_13_2025_ES_KinesinAB_RUES2_CAG72_KO_10ms_analyze'
output_directory = 'D:/4_26_2025_Kinesininneurons_cort_analyze'

# metadata_csv_path = 'D:/DENOISING/FromMetadata_test/metadata_summary.csv'
# metadata_csv_path = 'D:/DENOISING/DarkFrameCorrectionTest/metadata_summary.csv'

metadata_csv_path = 'D:/SAVED/metadata_summaryrah.csv'


# dark_frame_directory = 'D:/Darkframes100ms'
dark_frame_directory = 'D:/Darkframes10ms'




# Initialize ROISelector with metadata-enabled selection
selector = ROISelector(
    input_directory=input_directory,
    output_directory=output_directory,
    roi_width=150,  # Use the width from your ROI metadata
    roi_height=150,  # Use the height from your ROI metadata
    split_tiff=False,  # Set as per your needs
    dark_frame_subtraction=True,  # Enable dark frame subtraction if needed
    dark_frame_directory=dark_frame_directory,
    save_dark_corrected=False,  # Save dark-corrected images
    ROI_from_metadata=False,  # Enable ROI selection from metadata
    percentile_correction=True,
    metadata_path=metadata_csv_path,  # Provide path to the metadata CSV file
    pmin=1
)

# Prepare folders and process conditions
selector.prepare_output_folders()
selector.process_conditions()


In [ ]:
# Initialize an empty list to store DataFrames
dataframes = []

# Walk through the directory structure
for dirpath, dirnames, filenames in os.walk(master):
    # print(dirpath)
    # print(dirnames)
    # print(filenames)
    for filename in filenames:
        # print(filename)
        if filename.endswith('_tracked.mat'):
            file_path = os.path.join(dirpath, filename)
            # print(file_path)
            df = spt.read_mat_file(file_path)
            # print(file_path)
            # Add a column for the condition and file identifier
            # condition = os.path.basename(os.path.dirname(os.path.dirname(file_path)))
            condition = os.path.basename(os.path.dirname(file_path))


            identifier = os.path.splitext(filename)[0]
            df['condition'] = condition
            df['filename'] = identifier
            #### If you want to get the ms from the condition name, use below ####
            # extract the part of the condition name after the last underscore
            # condition_name = condition.split('_')[-1]
            # # remove the ms from the condition name
            # condition_name = condition_name.replace('ms', '')
            # time_between_frames = int(condition_name) / 1000
            # print(time_between_frames)
            #### If you want to get the ms from the condition name, use above ####
            df = spt.add_microns_and_secs(df, pixelsize_microns, time_between_frames)
            dataframes.append(df)

# # Concatenate all DataFrames into a single DataFrame
df = pd.concat(dataframes, ignore_index=True)
df = spt.add_unique_id(df)


In [ ]:
df.unique_id.unique()

In [ ]:
# # Filter the short tracks out!
df_filtstubs = spt.filter_stubs(df, min_time=0.2) #0.6 for before, 0.2 now

In [ ]:
# just load filtstubs
df_filtstubs = pd.read_csv(saved_data + 'df_filtstubs.csv')

In [ ]:
# # Define parameters
# # list of possible thresholds
# # segment_length_thresholds = [0.95, 1.0, 1.05]



# segment_length_threshold = 1.0 # segment length threshold in um. 
# print(segment_length_threshold)
# min_track_length_seconds = 0.2  # Specify the minimum track duration in seconds
# # time_between_frames = config.TIME_BETWEEN_FRAMES  # From config

# # Run the cleaning function
# cleaned_df, removed_unique_ids, report = spt.clean_and_split_tracks(
#     df_filtstubs, 
#     segment_length_threshold=segment_length_threshold, 
#     min_track_length_seconds=min_track_length_seconds,
#     time_between_frames=time_between_frames
# )

# # saved the cleaned df
# cleaned_df.to_csv(saved_data + 'cleaned_df.csv', index=False)

In [ ]:
# save the df_filtstubs
df_filtstubs.to_csv(saved_data + 'df_filtstubs.csv', index=False)

In [ ]:
pathfix_df = df_filtstubs.copy()

In [ ]:
# Define parameters
# list of possible thresholds
# segment_length_thresholds = [0.95, 1.0, 1.05]



segment_length_threshold = 1.0 # segment length threshold in um. 
# segment_length_threshold = 0.3

print(segment_length_threshold)
min_track_length_seconds = 0.2  # Specify the minimum track duration in seconds
# time_between_frames = config.TIME_BETWEEN_FRAMES  # From config

# Run the cleaning function
pathfix_df, pathfix_removed_unique_ids, pathfix_report = spt.pathfixer(
    df_filtstubs, 
    segment_length_threshold=segment_length_threshold, 
    min_track_length_seconds=min_track_length_seconds,
    time_between_frames=time_between_frames
)

# saved the cleaned df
pathfix_df.to_csv(saved_data + 'cleaned_df.csv', index=False)

In [ ]:
# NEW VERSION:

# Create an instance of ParticleMetrics with your data and time_between_frames value
particle_metrics = spt.ParticleMetrics(pathfix_df, time_between_frames)

# Calculate all features, without calculating time-windowed metrics initially
metrics_df = particle_metrics.calculate_all_features(calculate_time_windowed=False)

# Specify the window size and overlap (in number of frames)
window_size = 60  # e.g. 60 frames per window
overlap = 30      # e.g. 30 frames overlap between windows

# Calculate the time-windowed metrics using the updated method with extra options:
# - use_bounds: whether to enforce parameter bounds in the MSD fit (True)
# - max_D: maximum allowed diffusion coefficient (here, 5)
# - allow_partial_window: whether to process windows smaller than window_size (False)
# - min_window_size: minimum required frames per window (set to window_size)
particle_metrics.calculate_time_windowed_metrics(window_size=window_size,
                                                   overlap=overlap,
                                                   use_bounds=True, # (applying an upper bound for D)
                                                   max_D=5,
                                                   allow_partial_window=False, # (skip windows that have fewer frames than the specified window size)
                                                   min_window_size=window_size)

# Retrieve the DataFrames
time_averaged_df = particle_metrics.get_time_averaged_df()
time_windowed_df = particle_metrics.get_time_windowed_df()
msd_lagtime_df = particle_metrics.msd_lagtime_df

# Optionally, inspect the outputs
print("Time-Averaged DataFrame:")
display(time_averaged_df.head())
print("Time-Windowed DataFrame:")
display(time_windowed_df.head())
print("MSD Lag Time DataFrame:")
display(msd_lagtime_df.head())


In [ ]:
# Create an instance of ParticleMetrics
particle_metrics = spt.ParticleMetrics(pathfix_df, time_between_frames)

# Calculate all features, without calculating time-windowed metrics initially
metrics_df = particle_metrics.calculate_all_features(calculate_time_windowed=False)

# Specify the window size and overlap
window_size = 60#60#60  # for example, 20 frames
overlap = 30#30#30  # for example, 10 frames

# Calculate the time-windowed metrics with the specified window size and overlap
particle_metrics.calculate_time_windowed_metrics(window_size=window_size, overlap=overlap)

# Retrieve the DataFrames
time_averaged_df = particle_metrics.get_time_averaged_df()
time_windowed_df = particle_metrics.get_time_windowed_df()
msd_lagtime_df = particle_metrics.msd_lagtime_df


In [ ]:
# save those dfs
time_averaged_df.to_csv(saved_data + 'time_averaged_df.csv', index=False)
time_windowed_df.to_csv(saved_data + 'time_windowed_df.csv', index=False)
msd_lagtime_df.to_csv(saved_data + 'msd_lagtime_df.csv', index=False)
metrics_df.to_csv(saved_data + 'metrics_df.csv', index=False)




In [ ]:
time_averaged_df

In [ ]:
time_windowed_df

In [ ]:
metrics_df

In [ ]:
metrics_df_motion = spt.add_motion_class(metrics_df, time_windowed_df, time_window = 60)#60))
metrics_df_motion.to_csv(saved_data + 'metrics_df_motion.csv', index=False)



In [ ]:
# immediately cut out the unlabelled rows, because they ALWAYS occur at the end of tracks, where windows aren't big enough to get a D or an Alpha

print(f'Before: {len(metrics_df_motion)}')
metrics_df_motion_filt = metrics_df_motion[metrics_df_motion.motion_class != 'unlabeled']
print(f'After: {len(metrics_df_motion_filt)}')

print('These rows are always at the end of tracks, where windows are too small to get a D or an Alpha')
# save that also
metrics_df_motion_filt.to_csv(saved_data + 'metrics_df_motion_filt.csv', index=False)




In [ ]:
# read in the dfs
time_averaged_df = pd.read_csv(saved_data + 'time_averaged_df.csv')
time_windowed_df = pd.read_csv(saved_data + 'time_windowed_df.csv')
msd_lagtime_df = pd.read_csv(saved_data + 'msd_lagtime_df.csv')
metrics_df = pd.read_csv(saved_data + 'metrics_df.csv')
metrics_df_motion = pd.read_csv(saved_data + 'metrics_df_motion.csv')
metrics_df_motion_filt = pd.read_csv(saved_data + 'metrics_df_motion_filt.csv')


In [ ]:
# test the rest mate

metrics_df_motion_filt.unique_id.unique()

In [ ]:
# only the GC
location = 'GC'
metrics_df_location = metrics_df[metrics_df['Location'] == location]
time_windowed_df_location = time_windowed_df[time_windowed_df['Location'] == location]

In [ ]:
metrics_df_motion.condition.unique()

In [ ]:
# go through cells, make sure they look OK.
metrics_df_motion_filt_condition = metrics_df_motion_filt[metrics_df_motion_filt['condition'] == 'Condition_20H20S_cort']

In [ ]:
for cell in metrics_df_motion_filt_condition.filename.unique():
    print(cell)
    metrics_df_cell = metrics_df_motion_filt_condition[metrics_df_motion_filt_condition['filename'] == cell]
    # time_windowed_df_cell = time_windowed_df[time_windowed_df['cell'] == cell]
    spt.napari_visualize_image_with_tracksdev2(metrics_df_cell, condition=None,
                                                    cell=None, location=None, save_movie_flag=False)#master_dir=config.MASTER + 'data', condition=None, cell=None feature='Track_ID'



In [ ]:
spt.napari_visualize_image_with_tracksdev2(metrics_df_motion_filt, condition=None,
                                                    cell=None, location=None, save_movie_flag=False)#master_dir=config.MASTER + 'data', condition=None, cell=None feature='Track_ID'

In [ ]:
# okay so first, check the freehalo ones. Are they complete nonsense?

metrics_df_motion_filt.condition.unique()

In [ ]:
spt.napari_visualize_image_with_tracksdev2(metrics_df, condition='Condition_freehalo_cort',
                                                    cell=None, location=None, save_movie_flag=False)#master_dir=config.MASTER + 'data', condition=None, cell=None feature='Track_ID'

In [ ]:
metrics_df_motion.motion_class.unique()

In [ ]:
# make a df of only the unlabeled ones
unlabelled_df = metrics_df_motion[metrics_df_motion_filt.motion_class == 'unlabeled']

In [ ]:
# Group by unique_id and get the motion classes for each unique_id
grouped = metrics_df_motion.groupby('unique_id')['motion_class'].unique()

# Filter unique_ids that have both 'unlabeled' and any other class
unique_ids_with_unlabeled_and_others = grouped[grouped.apply(lambda x: 'unlabeled' in x and len(x) > 1)].index

# Convert to a list if needed
unique_ids_with_unlabeled_and_others = unique_ids_with_unlabeled_and_others.tolist()

print(unique_ids_with_unlabeled_and_others)



In [ ]:
# compare a given unique ID that HAS unlabelled in it, with one that doesn't
# unique ids with unlabelled
# make a df of only the unlabeled

# unique_id_unlabelled = metrics_df_motion[metrics_df_motion.motion_class == 'unlabeled'].unique_id.unique()


# pick a random one with numpy random
unique_id_unlabelled_random = np.random.choice(unique_ids_with_unlabeled_and_others)
# get a single particle df for that unique id from the metrics df_motion and the metrics_df_motion_filt
single_particle_df_unlabelled = metrics_df_motion[metrics_df_motion.unique_id == unique_id_unlabelled_random]
single_particle_df_unlabelled_filt = metrics_df_motion_filt[metrics_df_motion_filt.unique_id == unique_id_unlabelled_random]

In [ ]:
single_particle_df_unlabelled.frame.min()

In [ ]:
for frame in single_particle_df_unlabelled.frame.unique():
    print(f'For frame {frame}, the motion class is {single_particle_df_unlabelled[single_particle_df_unlabelled.frame == frame].motion_class.unique()}')
    # the length of the track is
print(f'The length of the track is {len(single_particle_df_unlabelled)}')

In [ ]:
# ok so what unique id is this?
unique_id_unlabelled_random

In [ ]:
# in the time windowed df, how many windows are there
# get a single particle df from the time windowed df
single_particle_df_unlabelled_time_windowed = time_windowed_df[time_windowed_df.unique_id == unique_id_unlabelled_random]

In [ ]:
single_particle_df_unlabelled_time_windowed

In [ ]:
single_particle_df_unlabelled_filt.frame.max()

In [ ]:
metrics_df.condition.unique()

In [ ]:
spt.napari_visualize_image_with_tracksdev2(metrics_df, condition='Condition_20H20S_cort', cell=None, location='GC', save_movie_flag=False, feature='particle', steps=99)

In [ ]:
spt.napari_visualize_image_with_tracksdev2(metrics_df_motion_filt, condition='Condition_20H20S_cort', cell=None, location='NN', save_movie_flag=True, feature='particle', steps=99)

In [ ]:
spt.napari_visualize_image_with_tracksdev2(metrics_df, condition='Condition_freehalo_cort', cell='loc-NN_type-cort_freehalo_005_cropped_tracked', location='NN', save_movie_flag=True, feature='particle', steps=99)

In [ ]:
# Beautiful coloured plots of tracks in a single cell

tracks_df = spt.plot_tracks_static(
    metrics_df_motion_filt,
    filename=None,
    file_id=None,
    location=None,
    condition=None,
    time_start=None, # in seconds
    time_end=None,
    # color_by='motion_class',
    color_by='motion_class',
    motion_type=None,
    overlay_image=False,

    scale_bar_length=2,
    scale_bar_position=(0.9, 0.1),
    scale_bar_color='white',
    transparent_background=True,
    save_path=spt.config.MASTER + 'static_trackplots/',
    display_final_frame=True,
    max_projection=False,
    contrast_limits=None,  # Tuple: (lower, upper) or None for auto
    invert_image=False,

    gradient = False,  # Frame interval in seconds
    colorway = 'Dark2',
    order = ['subdiffusive', 'normal', 'superdiffusive'],
    # figsize_multiplier = 1,
    dpi=200 ) # Multiplier for DPI to increase resolution)

In [ ]:
time_windowed_df.condition.unique()

In [ ]:
# make a df of only certain conditions
includedconditions = ['Condition_20H20S_cort', 'Condition_77H20S_cort', 'Condition_20H77S_cort']
time_windowed_df_included = time_windowed_df[time_windowed_df['condition'].isin(includedconditions)]

In [ ]:
time_windowed_df_included_location = time_windowed_df_included[time_windowed_df_included['Location'] == 'GC']

In [ ]:
# get the median avg_speed_um_s per filename for each condition and put it in a new dataframe
time_windowed_df_included_median = time_windowed_df_included_location.groupby(['filename', 'condition'])['avg_speed_um_s'].median().reset_index()
time_windowed_df_included_median


In [ ]:
time_windowed_df_included_location.columns

In [ ]:
# get the median avg_speed_um_s per filename for each condition and put it in a new dataframe
time_windowed_df_included_median = time_windowed_df_included_location.groupby(['filename', 'condition'])['diffusion_coefficient'].median().reset_index()
time_windowed_df_included_median


In [ ]:
feature = 'diffusion_coefficient'#'avg_speed_um_s' #'anomalous_exponent'
x_category = 'condition'
# order = orderofconditions
# order = ['CB','NN','GC']
order = includedconditions

spt.plot_boxplots(time_windowed_df_included_median, feature, x_category, font_size=22, order=order, palette='colorblind', 
                background='white', transparent=True, line_color='white', show_plot=True, 
                master_dir=None, grid=True, bw=False, strip=True, y_max=None, figsize=(8, 12), 
                annotate_median=True, rotation=90, dotsize = 10)

In [ ]:
feature = 'avg_speed_um_s'#'avg_speed_um_s' #'anomalous_exponent'
x_category = 'condition'
# order = orderofconditions
# order = ['CB','NN','GC']
order = includedconditions

spt.plot_boxplots(time_windowed_df_included_median, feature, x_category, font_size=22, order=order, palette='colorblind', 
                background='white', transparent=True, line_color='white', show_plot=True, 
                master_dir=None, grid=True, bw=False, strip=True, y_max=None, figsize=(8, 12), 
                annotate_median=True, rotation=90, dotsize = 10)

In [ ]:
# Gallery of tracks by motion class

spt.plot_tracks_by_motion_class(
    time_windowed_df, 
    metrics_df, 
    num_tracks=40, 
    colormap='Dark2', 
    axis_range=None, 
    show_annotations=False, 
    order=['subdiffusive', 'normal', 'superdiffusive'], 
    transparent_background=True, 
    annotation_color="white",
    text_size=16, 
    figsizemultiplier=5,  # Overall figure size multiplier for adaptable subplot size

)

In [ ]:
time_windowed_df.condition.unique()

In [ ]:
conditionsorder = ['Condition_10seconds','Condition_5seconds',]

In [ ]:
time_windowed_df.filename[0]

In [ ]:
# write a new function that adds a new column called 'cellID' to each row, based on the bit of the filename that is after 'cell-' and before '_cropped'
# this will be used to identify the cell in the napari viewer
def add_cellID(df):
    df['cellID'] = df.filename.str.extract(r'cell-(\d+)_cropped')
    return df
# also make a function that does this for the bit after 'power-' and before '_cell'
def add_powerID(df):
    df['powerID'] = df.filename.str.extract(r'power-(\d+)_cell')
    return df

In [ ]:
def add_laserpower(df):
    df['laserpower'] = df.condition.str.extract(r'power-(\d+)_cell')
    return df

In [ ]:
testdf.cellID.unique()

In [ ]:
test2df = add_powerID(testdf)

In [ ]:
test2df.powerID.unique()

In [ ]:
import pandas as pd
import re

def extract_metadata(df, filename_col):
    """
    Extracts 'power' and 'cellID' from the filename column and adds them as new columns.
    
    Args:
        df (pd.DataFrame): DataFrame containing a column with filenames.
        filename_col (str): Name of the column containing filenames.
    
    Returns:
        pd.DataFrame: Updated DataFrame with 'power' and 'cellID' columns.
    """
    def parse_filename(filename):
        match = re.search(r'power-(\d+percent)_cell-(\d+)', filename)
        if match:
            power = match.group(1)  # Extracts power (e.g., '25percent')
            cellID = int(match.group(2))  # Extracts cell number as integer
            return power, cellID
        return None, None  # If no match, return None values

    df[['power', 'cellID']] = df[filename_col].apply(lambda x: pd.Series(parse_filename(x)))

    return df

# Example usage:
# df = extract_metadata(df, 'filename')


In [ ]:
df = extract_metadata(time_windowed_df, 'filename')

In [ ]:
df

In [ ]:
chosencondition = 'Condition_10seconds'
#filter the df
df_filtered = df[df['condition'] == chosencondition]


In [ ]:
conditionsorder = ['25percent', '50percent']

In [ ]:
cell_df = df_filtered[df_filtered['cellID'] == 7]
feature = 'avg_speed_um_s'#'avg_speed_um_s' #'anomalous_exponent'
x_category = 'power'
# order = orderofconditions
# order = ['CB','NN','GC']
order = conditionsorder

spt.plot_boxplots(cell_df, feature, x_category, font_size=18, order=order, palette='colorblind', 
                background='white', transparent=True, line_color='white', show_plot=True, 
                master_dir=None, grid=True, bw=False, strip=True, y_max=None, figsize=(10, 12), 
                annotate_median=True, rotation=90, dotsize = 3)

In [ ]:
# conditionsorder = 

In [ ]:
feature = 'avg_speed_um_s'#'avg_speed_um_s' #'anomalous_exponent'
x_category = 'condition'
# order = orderofconditions
# order = ['CB','NN','GC']
order = orderofconditions

spt.plot_boxplots(time_windowed_df, feature, x_category, font_size=18, order=order, palette='colorblind', 
                background='white', transparent=True, line_color='white', show_plot=True, 
                master_dir=None, grid=True, bw=False, strip=False, y_max=None, figsize=(10, 12), 
                annotate_median=True, rotation=90, dotsize = 10)

In [ ]:
# # time to plot first segment length

# feature = 'segment_len_um'

# # NEW SHIT

# spt.plot_histograms(time_windowed_df, feature, bins=100, separate=None, xlimit=None, small_multiples=False, palette='colorblind',
#                     use_kde=False, show_plot=True, master_dir=None, tick_interval=5, average='mean', order=None, 
#                     grid=False, background='white', transparent=False, line_color='black', font_size=9, showavg=True,
#                     export_format='png', return_svg=False, x_range=None, y_range=None, percentage=True, 
#                     log_scale=False, log_base=10, alpha=1, log_axis_label='log', save_folder=None, figsize=(3,3))

In [ ]:
feature = 'diffusion_coefficient'
save_folder = 'A:/mshannon/2025/March/FIGURE5/histo/'

spt.plot_histograms(time_windowed_df, feature, bins=100, separate='condition_short', xlimit=None, small_multiples=True, palette='colorblind',
                    use_kde=True, kde_fill = True, show_plot=True, master_dir=None, tick_interval=2, average='median', 
                    order=dynein_conds,# here, you use the ORDER thing to define what is plotted in the histo :)
                    grid=False,  condition_colors=condition_colors, background='white', transparent=True, line_color='black', font_size=7, showavg=False,
                    export_format='svg', return_svg=False, 
                    x_range=[-4,2],
                      y_range=[0,0.7], 
                      percentage=True,log_scale=True, log_base=10, alpha=0.4, log_axis_label='actual', save_folder=save_folder, figsize=(5,3))

In [ ]:
spt.plot_classification_pie_charts(time_windowed_df, group_by='condition', colormap_name='Dark2', order = ['subdiffusive', 'normal', 'superdiffusive'], figsize=(30, 20), font_size=24, label_font_size=22)

In [ ]:
# remove unlabeled!
time_windowed_df = time_windowed_df[time_windowed_df.motion_class != 'unlabeled']

In [ ]:
# number of rows per condition
time_windowed_df.groupby('condition').count()

In [ ]:

x_category = 'condition'
# order = orderofconditions
# order = ['CB','NN','GC']

spt.plot_stacked_bar(time_windowed_df, x_category, order=order, font_size=12, colormap='Dark2', figsize=(10, 10), 
                     background='white', transparent=True, line_color='white')

In [ ]:
x_var = 'avg_speed_um_s'
y_var = 'n_frames'

spt.plot_combo_hist_scatter_kde(time_averaged_df, x_var, y_var, font_size=12, palette='mako', scatter_color=".15", hist_bins=50, kde_levels=5, 
                                figsize=(6, 6), separate=None, order=None, x_min=None, x_max=None, y_min=None, y_max=None, horizontal=False)

In [ ]:
chosenID = np.random.choice(cleaned_df['unique_id'].unique())

pathfix_single = spt.extract_single_particle_df(pathfix_df, unique_id = chosenID)
cleaned_single = spt.extract_single_particle_df(cleaned_df, unique_id = chosenID)

In [ ]:
pathfix_single

In [ ]:
# cleaned_single = spt.extract_single_particle_df(cleaned_df, unique_id = chosenID)
cleaned_single

In [ ]:
# alright, the unique ids have no .0 on the end for the pathfix_single
# This is good! Just check it works with the rest

In [ ]:
pathfix_single = spt.extract_single_particle_df(pathfix_df)
pathfix_single

In [ ]:
df_filtstubs

In [ ]:
# Define parameters
# list of possible thresholds
# segment_length_thresholds = [0.95, 1.0, 1.05]



segment_length_threshold = 1.0 # segment length threshold in um. 
print(segment_length_threshold)
min_track_length_seconds = 0.2  # Specify the minimum track duration in seconds
# time_between_frames = config.TIME_BETWEEN_FRAMES  # From config

# Run the cleaning function
pathfix_df, pathfix_removed_unique_ids, pathfix_report = spt.pathfixer(
    df_filtstubs, 
    segment_length_threshold=segment_length_threshold, 
    min_track_length_seconds=min_track_length_seconds,
    time_between_frames=time_between_frames
)

# saved the cleaned df
# pathfix_df.to_csv(saved_data + 'cleaned_df.csv', index=False)

In [ ]:
spt.visualize_track_changes_with_filtering(
    df_filtstubs, pathfix_df, pathfix_removed_unique_ids,
    filename=None,  
    time_start=25, time_end=27, 
    time_between_frames=0.1, 
    plot_size_px=150, 
    dpi=100,
    pixel_size_um=0.1,
    figsize=(18, 6),  
    line_width=1.2,  
    alpha_range=(0.3, 1.0),  
    transparent_background=True,
    overlay_image=True,  
    master_dir=master,  
    condition=None,  
    max_projection=True,  
    display_final_frame=False,  
    contrast_limits=None,  
    invert_image=True  
)

In [ ]:
# GOOD! 